In [24]:
import requests 
import json
import os 
# api-endpoint 
# location given here 
location = "Denver+CO"
try:
    api_key = open("key.txt").read()
    if (len(api_key)<2):
        print("Error: Key.txt file is empty ..")
except IOError:
      print ("Error: Key.txt File does not appear to exist.")

#Define the main parameters 
PARAMS = {'address':location, 'pageSize': 40, 'api_key': api_key} 
PARAMS

{'address': 'Denver+CO',
 'api_key': 'dckzqqFYqhjvUiEQ9XjUDhQEmckxhwXOkb0HeTZF',
 'pageSize': 40}

In [36]:
#Retrieves a paged list of foods. Use PageSize to set the Maximum number of results to return for the current page. Default is 50
def getListofFoodJSON(PARAMS):
    URL = "https://api.nal.usda.gov/fdc/v1/foods/list"
    #sending get request and saving the response as response object 
    r = requests.get(url = URL, params = PARAMS) 
    # extracting data in json format 
    data = r.json() 
    return data

#Filter the list of food to keep the main information only
def ParseListofFood(list_):
    new_list = []
    main_info = ['dataType','fdcId','description']
    main_nuts = ['204','205','291','296','208','203'] #carbs, fiber, fat, energy and protein 
    for food in list_:
        new_food_dict = {k: v for k, v in food.items() if k in main_info}
        nuts_list = food['foodNutrients']
        new_nuts_list = []
        for food_nut in nuts_list:
            for key, value in food_nut.items():
                if key == "number" and value in main_nuts:
                    new_nuts_list.append(food_nut)
        new_food_dict['foodNutrients'] = new_nuts_list
        new_list.append(new_food_dict)
    return new_list   

#Fetches details for a food item using input FDC ID
def getReportJSON(fdcID,PARAMS,foodType=""):
    URL = "https://api.nal.usda.gov/fdc/v1/food/"+str(fdcID)
    r = requests.get(url = URL, params = PARAMS) 
    # extracting data in json format 
    data = r.json() 
    return data

def getReportsJSON(fdcIDs,PARAMS,foodType=""):
    URL = "https://api.nal.usda.gov/fdc/v1/foods"
    PARAMS['fdcIds'] = fdcIDs
    r = requests.get(url = URL, params = PARAMS) 
    # extracting data in json format 
    data = r.json() 
    return data

#Item: is a json object of a single food item 
def getPortionInfo(item):
    info = {}
    gramWieght = item['foodPortions'][0]['gramWeight']
    amount = item['foodPortions'][0]['amount']
    measureUnit_abb = item['foodPortions'][0]['measureUnit']['abbreviation']
    measureUnit_name = item['foodPortions'][0]['measureUnit']['name']
    measureUnit_id = item['foodPortions'][0]['measureUnit']['id']
    info['gramWieght'] = gramWieght
    info['amount'] = amount
    info['measureUnit_abb'] = measureUnit_abb
    info['measureUnit_name'] = measureUnit_name
    info['measureUnit_id'] = measureUnit_id
    return info
    

In [29]:
data = getListofFoodJSON(PARAMS)
print(type(data))
print(type(data[0]))

<class 'list'>
<class 'dict'>


In [39]:

data = getReportsJSON(keys,PARAMS)
data[0]
getPortionInfo(data[0])

{'amount': 1.0,
 'gramWieght': 223.0,
 'measureUnit_abb': 'cup',
 'measureUnit_id': 1000,
 'measureUnit_name': 'cup'}

### Build a pandas df to map measures information to IDs 
 Measures information includes: measure unit abbreviation, measure unit name and measure unit ID
 
 Note: this is only applicable to Foundation food dataset

In [38]:
import pandas as pd
#Map all measures to ids (measure abb, name , id)
measure_abbs = [] 
measure_names = []
measure_ids = []
df_dict = {}
for item in data:
    dict_ = getPortionInfo(item)
    measure_abbs.append(dict_['measureUnit_abb'])
    measure_ids.append(dict_['measureUnit_id'])
    measure_names.append(dict_['measureUnit_name'])
df_dict["name"] = measure_names
df_dict["abb"] = measure_abbs
df_dict["id"] = measure_ids

df = pd.DataFrame.from_dict(df_dict)
len(df)

35